In [1]:
import networkx as nx
import csv
import operator
from prettytable import PrettyTable

In [2]:
'''
CSV-Dateien einlesen
'''
ids = []   # Liste Ids zu Email-Adressen
emails = [] # Liste Email-Adressen
with open('ids.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    next(readCSV) #erste Zeile wird ignoriert
    
    for column in readCSV:
        ids.append(column[0]) # 1. Spalte der Tabelle sind die IDs
        emails.append(column[1]) # 2. Spalte der Tabelle sind die Email-Adressen
        
        

In [3]:
idsFrom = [] # Liste IDs der Absender der Emails
idsTo = [] # Liste IDs der Empfaenger der Emails
with open('dnc-temporalGraph.csv') as csvfile2:
    readCSV2 = csv.reader(csvfile2, delimiter='\t')
    next(readCSV2)
    for column in readCSV2:
        idsFrom.append(column[0])
        idsTo.append(column[1])

In [4]:
'''
Da manche Email-Adressen mehrfach vorkommen, sich nur in Gross- und Kleinschreibung unterscheiden, 
werden alle Email-Adressen in Kleinbuchstaben geschrieben gespeichert
'''

for i in range(0, len(emails)):
    emails[i] = emails[i].lower()

In [5]:
'''
Dictionary mit IDs als keys und Email-Adressen als values erstellen
'''

dictIdEmail = {}
for i in range(0, len(emails)):
    dictIdEmail.update({ids[i]:emails[i]})

In [7]:
'''
IDs in idsFrom und idsTo werden durch Email-Adressen ersetzt
'''
for i in range(0, len(idsFrom)):
    idsFrom[i] = dictIdEmail[idsFrom[i]]
    
for i in range(0, len(idsTo)):
    idsTo[i] = dictIdEmail[idsTo[i]]
    

In [8]:
'''
mehrfach vorkommende Email-Adressen sollen wie eine einzige Adresse behandelt werden,
deshalb werden diese entfernt
'''
emails2 = []
for i in range(0, len(emails)):
    if emails[i] not in emails2:
        emails2.append(emails[i])
emails = emails2


In [9]:
'''
Knoten in Graph G einfuegen
'''
G = nx.DiGraph()

for i in range(0, len(emails)):
    G.add_node(emails[i])

In [10]:
'''
Kanten in Graph G einfuegen
'''
for i in range(0, len(idsFrom)):
    G.add_edge(idsFrom[i], idsTo[i])

In [11]:
'''
Loeschen von Knoten ohne Email-Adressen, da diese von Gephi sowieso ignoriert werden.
Wuerden diese Knoten nicht geloescht werden, kann dies z.B. zu unterschiedlichen PageRank-Werten
fuehren, d.h. die PageRank-Werte welche mit Python berechnet werden unterscheiden sich von
den PageRank-Werten welche mit Gephi berechnet werden. (wollte eigentlich leeren Email-Adressen "unknown"
zuweisen, daber dann wurde ich dies ja korrigieren bzw. veraendern und nicht ignorieren)
'''
G.remove_node('')

In [13]:
'''
Selfloops entfernen
'''
selfloopList = list(nx.nodes_with_selfloops(G))
for i in range(0, len(selfloopList)):
    G.remove_edge(selfloopList[i], selfloopList[i])

In [14]:
'''
Bestimmen der groesseten zusammenhaengenden Komponente in Graph G,
welche anschliessend neuer Graph G wird
'''
largest = max(nx.strongly_connected_components(G), key=len)
G = G.subgraph(largest)

In [17]:
'''
15 Email-Adressen mit hoechsten Eingrad bestimmen und als Tabelle ausgeben
'''
listInDegree = []

for i in range(0, len(emails)):
    if G.__contains__(emails[i]):
        listInDegree.append((G.in_degree(emails[i]), emails[i]))
        
listInDegreeSorted = sorted(listInDegree, key=lambda list1: list1[0], reverse=True)

pretTab = PrettyTable(["EmailAdresse", "Eingrad"])
for i in range(0, 15):
    pretTab.add_row([listInDegreeSorted[i][1], listInDegreeSorted[i][0]])
print(pretTab)

+----------------------+---------+
|     EmailAdresse     | Eingrad |
+----------------------+---------+
|   mirandal@dnc.org   |   218   |
|   kaplanj@dnc.org    |   133   |
|    comers@dnc.org    |   100   |
| paustenbachm@dnc.org |    85   |
|   parrishd@dnc.org   |    68   |
|   walkere@dnc.org    |    64   |
|   palermor@dnc.org   |    52   |
|  manriquezp@dnc.org  |    51   |
|    allenz@dnc.org    |    50   |
|    daceya@dnc.org    |    48   |
|  houghtonk@dnc.org   |    43   |
|   garciaw@dnc.org    |    39   |
| freundlichc@dnc.org  |    39   |
|     weis@dnc.org     |    38   |
|   dillonl@dnc.org    |    37   |
+----------------------+---------+


In [20]:
'''
15 Email-Adressen mit dem hoechsten PageRank bestimmen und als TAbelle ausgeben
'''
pageRankDict = nx.pagerank(G)
pageRankSorted = sorted(pageRankDict.items(), key=operator.itemgetter(1), reverse=True)

pretTab2 = PrettyTable(["EmailAdresse", "PageRank"])
for i in range(0, 15):
    pretTab2.add_row([pageRankSorted[i][0], pageRankSorted[i][1]])
print(pretTab2)

+----------------------+----------------------+
|     EmailAdresse     |       PageRank       |
+----------------------+----------------------+
|   mirandal@dnc.org   | 0.07313211867573442  |
|   kaplanj@dnc.org    | 0.052427860188623415 |
|    comers@dnc.org    | 0.03489861769062445  |
|   parrishd@dnc.org   | 0.024705659364351307 |
| paustenbachm@dnc.org | 0.019163997032295967 |
|    allenz@dnc.org    | 0.01759766830666043  |
|    stowee@dnc.org    | 0.01219232586877108  |
|   walkere@dnc.org    | 0.011508173844306592 |
|   wrighta@dnc.org    | 0.011385481760180036 |
|    daceya@dnc.org    | 0.01059455092200594  |
|   palermor@dnc.org   | 0.010574923987952988 |
|  manriquezp@dnc.org  | 0.009837432974887785 |
|   hoffmana@dnc.org   | 0.009138771960749702 |
|  houghtonk@dnc.org   | 0.008569599731757527 |
|     coxc@dnc.org     |  0.0079664063918546  |
+----------------------+----------------------+


In [22]:
'''
Erstellen einer gexf_Datei des Graphen G, welche mit Gephi geoffnet werden kann
'''
nx.write_gexf(G, "Loesung_Graph_GroessteKomp.gexf")